In [3]:
import base64
import re

def decode_base64_in_svg(svg_filename):
    # Read the SVG file
    with open(svg_filename, 'r') as svg_file:
        svg_content = svg_file.read()

    # Define a regular expression to find base64 encoded strings
    base64_pattern = r'data:image/[^;]+;base64,([^"]+)'

    # Find all matches of base64 encoded strings in the SVG content
    base64_matches = re.findall(base64_pattern, svg_content)

    # Iterate over each match and decode the base64 data
    for base64_data in base64_matches:
        # Decode the base64 data
        decoded_data = base64.b64decode(base64_data)

        # Replace the base64 string with the decoded data in the SVG content
        svg_content = svg_content.replace(f'data:image;base64,{base64_data}', decoded_data.decode('utf-8'))

    # Return the SVG content with base64 data decoded
    return svg_content

# Example usage:
input_svg_filename = 'diagrams/finalqr.svg'
decoded_svg = decode_base64_in_svg(input_svg_filename)

# You can then save the decoded SVG content to a file or do further processing.
# with open('decoded_example.svg', 'w') as output_svg_file:
#     output_svg_file.write(decoded_svg)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

In [1]:
from configs import *
import numpy as np
import pandas as pd
import streamlit as st
import gspread
from google.oauth2 import service_account


In [2]:
def connect_to_gs(_service_account_key):
    scopes = ["https://www.googleapis.com/auth/spreadsheets"]
    credentials = service_account.Credentials.from_service_account_info(
        _service_account_key, scopes=scopes
    )
    gs_connection = gspread.authorize(credentials)
    return gs_connection

In [3]:
gs_connection = connect_to_gs(st.secrets["gcp_service_account"])

In [4]:
def fetch_swap_data(_gc, sheet_name, sheet_key, columns_list):
    try:
        # Open specific sheet
        gs = _gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        # to handle numeric columns that are imported as strings
        for column in columns_list:
            df[column] = pd.to_numeric(df[column])

        return df

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None


In [5]:
sd = fetch_swap_data(gs_connection, find_swap_table, prod_google_sheet_key, [])

In [8]:
def count_unique_values(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")
    
    unique_values = df[column_name].nunique()
    return unique_values

In [10]:
unique_users = count_unique_values(sd, "email")

In [11]:
unique_users

2

In [17]:
def count_grouped_values(df):
    if not all(col in df.columns for col in ['current_placement', 'first_choice', 'second_choice', 'third_choice']):
        raise ValueError("Required columns not found in the DataFrame.")
    
    placement_counts = df['current_placement'].value_counts()
    first_choice_counts = df['first_choice'].value_counts()
    second_choice_counts = df['second_choice'].value_counts()
    third_choice_counts = df['third_choice'].value_counts()
    
    return placement_counts, first_choice_counts, second_choice_counts, third_choice_counts


In [18]:
placement_counts, first_choice_counts, second_choice_counts, third_choice_counts = count_grouped_values(sd)

In [19]:
placement_counts

Addington Hospital    1
Boitumelo Hospital    1
Name: current_placement, dtype: int64

In [ ]:
df = categories(drinks)
domain = ['First Choice', 'Second Choice', 'Third Choice']
range_ = ['#c62828', '	#ef9a8d', '#777777', 'blue']

bar_chart = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        alt.X("Name:O", axis=alt.Axis(title="Name"), sort="-y"),
        alt.Y("sum(quantity):Q", axis=alt.Axis(title="Drinks")),
        alt.Order("sum(quantity):Q", sort="descending"),
        alt.Color(
            "Category:N",
            scale=alt.Scale(domain=domain, range=range_),
            legend=alt.Legend(orient="top", direction="horizontal", title=None),
        ),
    )
)
st.altair_chart(bar_chart, use_container_width=True)

In [20]:
def pivot_choices_to_new_df(df):
    if not all(col in df.columns for col in ['first_choice', 'second_choice', 'third_choice']):
        raise ValueError("Required columns not found in the DataFrame.")
    
    # Reshape the data to long format
    choices_df = pd.melt(df, value_vars=['first_choice', 'second_choice', 'third_choice'],
                         var_name='choice', value_name='hospital')
    
    return choices_df

In [21]:
s =  pivot_choices_to_new_df(sd)
s

,choice,hospital
0,first_choice,Dr George Mukhari Hospital
1,first_choice,Addington Hospital
2,second_choice,Charlotte Maxeke Hospital
3,second_choice,Dihlabeng Hospital
4,third_choice,Cofimvaba Hospital
5,third_choice,Dr George Mukhari Hospital
